# Projet Informatique - Comics to Music

## GANs non-conditionnel

Dans le code, en voici les différents éléments : 
 - définition du générateur
 - définition du discriminateur
 - entraînement du modèle

In [2]:
import torch
from torch import nn, optim
import torchvision
import torchvision.transforms as transforms
import math
import matplotlib.pyplot as plt
import os

In [3]:
device = ''
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

### Générateur (sans conditionnement)

In [4]:
class NonConditionnalGenerator(nn.Module):
    def __init__(self, z_dim):
        super().__init__()
        
        '''On commence par définir les éléments structurels de notre générateur.
        Dans notre cas, on suppose d'abord qu'on veut générer vers un espace latent
        (génération conditionnée par la suite par un autre espace latent, plus des
        conditions supplémentaires).
        Le générateur va augmenter la dimension des données en opérant une déconvolution (Upsample, 2e article cGANs)
        Ici, on part d'un vecteur bruit z et on utilise une première couche linéaire pour 
        le mettre sous format-image en 2D. Dans cet exemple, on veut obtenir des images 28x28, donc on transforme z 
        en images 7x7, ce qui permettra facilement d'atteindre la dimension finale en réalisant deux Upsample de scale = 2
        '''

        self.z_dim = z_dim # dimension du vecteur bruit 
        self.init_dim = 7 # dans cet exemple, on veut partir d'une image 7x7
        self.num_ch = 128 # nombre de canaux arbitraire (dépend de l'espace latent, peut être des autres conditions)
        self.l1 = nn.Sequential( # première couche linéaire
            nn.Linear(z_dim, self.num_ch * self.init_dim * self.init_dim) # on change la dimension du bruit pour pouvoir le 
                                                                          #reshape en [num_ch] images de dimensions init_dim x init_dim
        )

        # Réseau convolutionnel pour générer une image 28x28  : G_non_conditionnel : bruit --> espace latent audio
        self.model = nn.Sequential(
            nn.BatchNorm2d(self.num_ch),
            nn.Upsample(scale_factor=2), # passer de 7x7 à 14x14
            nn.Conv2d(self.num_ch, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2), # on passe à 28x28
            nn.Conv2d(256, 1, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )
        ######################
        
    def forward(self, x):
        # conversion du vecteur bruit
        output1 = self.l1(x) 
        # reshape en image
        output2 = output1.view(output1.shape[0], self.num_ch, self.init_dim, self.init_dim) 
        # CNN
        output = self.model(output2) 
        return output

In [5]:
z_dim = 100 # arbitraire
nc_generator = NonConditionnalGenerator(z_dim).to(device=device)

### Discriminateur (sans conditionnement)

In [6]:
class NonConditionnalDiscriminator(nn.Module):
    def __init__(self, num_ch):
        super().__init__()
        
        '''Le discriminateur va être entraîner pour différencier les données réelles (espace latent audio)
        des données générées (G(z), qui dans la version cGAN seront conditionnées par l'espace latent BD).
        En pratique, il ne prend ne prend en entrée que des vrais ou que des faux à la fois.
        Fonctionnement de l'entraînement : 
         - on génère les prédictions D(x_real) --> objectif D(x_real) = 1
         - on calcule la loss sur les prédictions réelles : loss_real = loss(D(x_real), 1)
         - on génère les prédictions D(x_fake) --> objectif D(x_fake) = 1
         - on calcule la loss sur les prédictions réelles : loss_fake = loss(D(x_fake), 1)
        Le discriminateur est donc entraîné sur loss_tot = (loss_real + loss_fake)
        
        Architecture : réseau convolutionnel classique, exemple dans le deuxième article sur les cGANs'''
        
        self.num_ch = num_ch # nombre de canaux, dépend de l'espace latent audio

        #  modèle de convolution classique, on cherche à densifier l'information des images à chaque couche
        self.model = nn.Sequential(
            nn.Conv2d(num_ch, 64, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), nn.BatchNorm2d(128), nn.LeakyReLU(0.2),
            nn.Conv2d(128, 512, kernel_size=3, stride=2, padding=1), nn.BatchNorm2d(512), nn.LeakyReLU(0.2),
            
            # A la fin, on réduit brusquement le nombre de canaux à 1, pour obtenir une prédiction scalire {0,1}.
            # Dans la version cGAN, c'est ici que l'on introduit la condition y !!!
            nn.Conv2d(512, 1, kernel_size=4, stride=2, padding=0),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        # prend un batch d'images d'entrée, sort un batch de 1 et de 0
        output = self.model(x).view(-1,1)
        return output

In [7]:
num_ch = 3 # arbitraire
nc_discriminator = NonConditionnalDiscriminator(num_ch).to(device=device)

### Entraînement (sans conditionnement)

In [8]:
# Dataset générés à partir de l'espace latent audio

batch_size = 32
train_loader = ???

SyntaxError: invalid syntax (844242149.py, line 4)

In [11]:
# Paramètres de l'entraînement

lr = 0.0001
num_epochs = 50
loss_function = nn.BCELoss() # ici, loss commune à D et G, à voir si on peut utiliser une loss différente

optimizer_nc_discriminator = torch.optim.Adam(nc_discriminator.parameters(), lr=lr)
optimizer_nc_generator = torch.optim.Adam(nc_generator.parameters(), lr=lr)

In [12]:
# Sauvegarde et récupération de G et D 
if os.path.isfile('discriminator2.pt') and os.path.isfile('generator2.pt'):
    nc_discriminator.load_state_dict(torch.load('./discriminator2.pt'))
    nc_generator.load_state_dict(torch.load('./generator2.pt'))   

else:
    for epoch in range(num_epochs):

        for n, (real_samples, _) in enumerate(train_loader):

            ### Entraînement nc_discriminator
            # On récupère les données réelles (reshape pour être sûr qu'elles soient acceptés par nc_D)
            real_samples = real_samples.view(-1, 1, 28, 28)

            # on génère les fake_samples à partir d'un bruit gaussien (pas de condition ici)
            z = torch.randn(batch_size, z_dim)
            fake_samples = torch.tanh(nc_generator(z))

            # Initialisation de la descente de gradient
            optimizer_nc_discriminator.zero_grad()

            # Loss sur les prédictions "données réelles"
            predict_real = nc_discriminator(real_samples)
            loss_real = loss_function(predict_real, torch.full((batch_size, 1), 1.0))

            # Loss sur les prédictions "données générées"
            predict_fake = nc_discriminator(fake_samples.detach())
            loss_fake = loss_function(predict_fake, torch.full((batch_size, 1), 0.0))

            # Loss complète
            loss_discriminator = (loss_real + loss_fake) / 2

            # Descente de gradient et mise à jour des poids w_D
            loss_discriminator.backward()
            optimizer_nc_discriminator.step()

            ### Entraînement nc_generator
            # Initialisation descente de gradient
            optimizer_nc_generator.zero_grad()

            # génération des fake_samples
            z = torch.randn(batch_size, 100)
            fake_samples_new = nc_generator(z)

            # récupération de la prédiction de D
            gen_prediction = nc_discriminator(fake_samples_new)

            # loss classique entre la prédiction D(fake_samples) et l'objectid vecteur de 1
            valid_labels = torch.full((batch_size, 1), 1.0)
            loss_generator = loss_function(gen_prediction, valid_labels)

            # OPTIONNEL --> diversity loss
            # on ajoute une diversity loss pour éviter de générer que des points identiques
            # sinon le générateur peut mapper à chaque fois vers une seule image qui à l'air vraie
            distances = torch.cdist(fake_samples_new, fake_samples_new, p=2) # renvoie la distance scalaire entre tous les éléments générés du batch
            mean_distance = torch.mean(distances)
            lambda_div = 0.2 # poids sur la loss
            loss_diversity = -lambda_div * mean_distance # on veut maximiser la distance donc on met un - devant

            # loss totale 
            loss_generator = loss_generator + loss_diversity
            
            loss_generator.backward()
            optimizer_nc_generator.step()

            # Show loss
            if n == batch_size - 2:
                print(f"Epoch: {epoch} Loss D.: {loss_discriminator}")
                print(f"Epoch: {epoch} Loss G.: {loss_generator}")

NameError: name 'train_loader' is not defined

## GANs conditionnels

Pas si différents des GANs non-conditionnels, il faut juste rajouter une condition $y$ à deux étapes du modèle.

In [13]:
class ConditionnalGenerator(nn.Module):
    def __init__(self, z_dim, y_dim):
        super().__init__()
        
        self.z_dim = z_dim # dimension du vecteur bruit 
        self.y_dim = y_dim # dimension du vecteur condition
        self.init_dim = 7 # dans cet exemple, on veut partir d"'une image 7x7
        self.num_ch = 128 # nombre de canaux arbitraire (dépend de l'espace latent, peut être des autres conditions)
        
        self.l1 = nn.Sequential( # première couche linéaire
            nn.Linear(self.z_dim + self.y_dim, self.num_ch * self.init_dim * self.init_dim) # cette fois, on part de 
                                                                                            # z_dim + y_dim
        )

        # Réseau convolutionnel identique au vanilla GAN
        self.model = nn.Sequential(
            nn.BatchNorm2d(self.num_ch),
            nn.Upsample(scale_factor=2), # passer de 7x7 à 14x14
            nn.Conv2d(self.num_ch, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2), # on passe à 28x28
            nn.Conv2d(256, 1, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )
        ######################
        
    def forward(self, x):
        # Ici, x est la concaténation du bruit z et de la condition y
        # conversion du vecteur x
        output1 = self.l1(x) 
        # reshape en image
        output2 = output1.view(output1.shape[0], self.num_ch, self.init_dim, self.init_dim) 
        # CNN
        output = self.model(output2) 
        return output

In [14]:
z_dim = 100 # arbitraire
y_dim = 100 # arbitraire
c_generator = ConditionnalGenerator(z_dim, y_dim).to(device=device)

In [15]:
class ConditionnalDiscriminator(nn.Module):
    def __init__(self, num_ch, latent_x_dim, y_cond_dim):
        super().__init__()
    
        self.num_ch = num_ch # nombre de canaux, dépend de l'espace latent audio
        self.y_cond_dim = y_cond_dim
        self.stride_size = 2 # valeur du stride pour connaître la dimension lors du reshape

        #  modèle de convolution classique, on cherche à densifier l'information des images à chaque couche
        self.model = nn.Sequential(
            nn.Conv2d(num_ch, 64, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), nn.BatchNorm2d(128), nn.LeakyReLU(0.2),
            nn.Conv2d(128, 512, kernel_size=3, stride=2, padding=1), nn.BatchNorm2d(512), nn.LeakyReLU(0.2),
        )

        # A cette étape, il faut applatir les données et y concaténer la condition y
        # calcul de la dimension du reshape :
        self.image_dim = int(latent_x_dim / (self.stride_size ** 3)) # on a appliqué 3 strides de 2
        self.flat_dim = 512 * self.image_dim * self.image_dim

        self.classifier = nn.Sequential(
            nn.Linear(self.flat_dim + self.y_cond_dim, 1), # ce n'est plus une convolution car vecteur plat
            nn.Sigmoid()
        )

    
    def forward(self, x, y):
        # on rentre x et y car on commence par traiter seulement x avant d'injecter y
        features = self.model(x)
        
        # Aplatissement vers la flat dimension
        features_flat = features.view(features.size(0), -1) 
        
        # Concaténation avec le vecteur condition y
        combined = torch.cat([features_flat, y], dim=1)
        
        # Prédiction finale
        output = self.classifier(combined)
        return output


In [16]:
num_ch = 3 # arbitraire
x_dim = 16 # arbitraire
y_cond_dim = 28 # arbitraire
c_discriminator = ConditionnalDiscriminator(num_ch, x_dim, y_cond_dim).to(device=device)

## GANs Cycle-Consistent

Dans notre cas, on veut que le GAN apprennent à générer des données dans l'espace latent $Y$ (audio) en étant influencé par l'espace latent $X$ (BDs). Pour cela, deux solutions : entraîner notre GAN avec des données réelles pairées, en conservant les paires dans les espaces latents $X$ et $Y$. Problème : auncun dataset existant possédant des paires d'audio et de BDs. On va donc chercher à entraîner notre GAN pour qu'il puisse associer des échantillons $x\in \{x_i\}$ à des échantillons $y\in\{y_i\}$, de manière non supervisée, en détectant lui-même les caractéristiques pour associer une paire. On s'inspire de l'article *Unpaired Image-to-Image Translation
using Cycle-Consistent Adversarial Networks*, qui introduisent les *Cycle-Consistent GANs*. Ils utilisent ce réseau pour passer d'images à images. Dans notre cas, les espaces $X$ et $Y$ ne sont pas aussi similaires, il va peut être falloir injecter des conditions supplémentaires (structure, rythme, couleur, texte, sens --> CLIP), pour aider notre GAN à générer des paires qui ont du sens.

En pratique, on a besoin d'entraîner deux générateurs
$$
G : X \longrightarrow Y \\
F : Y \longrightarrow X
$$

Chacun associé à un discrimnateur, qui va déterminer si la sortie du générateur correspond à l'espace réel cible. Là génération est donc entraînée par une adversarial loss. On s'inspire des travaux de *Improved Training of Wasserstein GANs* qui ont introduit les WGANs, utilisant une distance de Wasserstein comme adversarial loss. 

$$
\begin{align}
\mathcal{L}_{WGAN} (G, D_Y, X, Y) =\: &\mathbb{E}_{y\sim p_{data}(y)}[D_Y(y)]\\ - \: &\mathbb{E}_{x\sim p_{data}(x)}[D_Y(G(x))]
\end{align}
$$ 

Contrairement à un GAN classique, on ne cherchera à prédire des 1 ou des 0. On va ici chercher à maximiser les écarts entre les score vrais ou faux. Nos discriminateurs sortent donc des vecteurs de score, et non pas des labels de prédiction. On en tire une loss pour les discriminateurs, et une pour les générateurs : 
$$
\begin{align}
\mathcal{L}_D &= \mathbb{E}[D(G(x))] - \mathbb{E}[D(y)] + \lambda_{GP}\mathcal{L}_{GP}\\
\mathcal{L}_G &= -\mathbb{E}[D(G(x))]
\end{align}
$$

où $\mathcal{L}_{GP}$ est la $gradient \: penalty$ introduite dans l'article WGANs (ils utilisent $\lambda_{GP}=10$). Cette pénalité vise à empêcher la sortie du discriminateur d'avoir une norme différente de 1. Ils l'expriment comme : 
$$
\mathcal{L}_{GP} = \mathbb{E}[(||\nabla_{\hat{y} = G(x)}D(G(x))||_2 -1)^2]
$$
Un aspect important de l'article est qu'il stipule que le discriminateur doit être entraîné plus souvent que le générateur. Environ 5 steps pour 1.


L'architecture globale *Cycle-GANs* nécessite d'ajouter une *cycle-loss*, qui détermine si les données sont bien reconstruites lors de l'application successive de $G$ et $F$ :
$$
\begin{align}
\mathcal{L}_{cyc} =\: &\mathbb{E}_{x\sim p_{data}(x)}[||F(G(x)) - x||]\\ + \:&\mathbb{E}_{y\sim p_{data}(y)}[||G(F(y)) - y||]
\end{align}
$$


Pour l'architecture, l'article *Cycle-GANs* s'inspire de Johnson et al. *Perceptual losses
for real-time style transfer and super-resolution*. On y ajoute la notion développée dans l'article *LOGAN : Unpaired Shape Transform in Latent Overcomplete Space*, qui présente l'avantage de travailler avec des espaces *overcomplete* (réseaux dans lesquels la dimension a été fortement augmentée). Cela facilite le transport de masse optimale entre les deux distributions (--> coût égal *masse* x *distance(X,Y)*). Cet ajout agit avec la Wassertein-Loss pour empêcher le *mode-collapse* : 

1. Générateurs :
 - $\longrightarrow$ donc pour ces GANs là, on n'augmente pas forcément la dimension comme vu dans les cGANs (on peut quand même modifier si jamais nos espaces $X$ et $Y$ n'ont pas les mêmes dimensions). Nos espaces latents auront peut être des dimensions différentes (par exemple : peut être besoin de plus grandes dimensions dans l'espace latent audio, si on utilise plein de conditions supplémentaires dans l'espace BD). 

 2. Discriminateurs :
 - $N\times N$ Patch-GANs : faits pour discriminer des grosses images (1080p par exemple). Ils ne discriminent que sur des portions de $N$ par $N$. Peut être pas nécessaire pour nous si on travaille sur des espaces latents de faibles dimensions. Mais peux justement permettre de travailler sur des espaces latents plus complexes sans trop augmenter la charge de calcul.

In [17]:
def compute_gradient_penalty(D, real_samples, fake_samples, device=device):
    """Calcul du Gradient Penalty pour WGAN-GP"""
    
    # On adapte les dimensions de alpha pour qu'elles collent aux données (1D ou 2D)
    alpha = torch.rand(real_samples.size(0), 1, 1, 1, device=device) if len(real_samples.shape) == 4 else torch.rand(real_samples.size(0), 1, device=device)
    
    # si c'est un vecteur 1D (X), alpha doit être (Batch, 1)
    # si c'est un tensuer 2D (Y), alpha doit être (Batch, 1, 1, 1) pour le broadcast
    if len(real_samples.shape) == 2:
         alpha = alpha.view(real_samples.size(0), 1)
    
    # Interpolation
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    
    # Passage dans le discriminateur
    d_interpolates = D(interpolates)
    
    # Calcul du gradient
    # On crée un vecteur de 1 --> cible de notre gradient
    fake = torch.ones(d_interpolates.size(), device=device, requires_grad=False)
    
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    
    # Calcul de la pénalité
    gradients = gradients.view(gradients.size(0), -1) # on aplatit pour le calcul de norme
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    
    return gradient_penalty

### Générateur $G$ : $X$(BDs) $\longrightarrow$ $Y$(audios)

$G$ part de $X$ l'espace latent BDs pour générer $Y$ l'espace latent audio. Comment on utilisera des conditions supplémentaires pour influencer la génération, on peut se permettre de ne garder que peut d'information dans $X$. Ainsi, on choisit de générer $X$ (partie VAE) comme un vecteur 1D plutôt que comme une image. On ne garde ainsi qu'une idée du style et on oublie les informations de structure spatiale qui seront utilisées plus efficacement dans les autres conditions.

In [18]:
class Generator_G(nn.Module):
    def __init__(self, z_dim, latent_x_dim, x_ch, init_dim):
        '''
        Inputs :
         - z_dim : dimension du vecteur bruit seed du générateur
        Dimensions des espaces latents: 
         - latent_x_dim : dimension de l'espace latent X d'où est tirée la condition du générateur G.
            On peut générer l'espace latent BDs comme un vecteur, car on ne veut l'utiliser que pour avoir une 
            notion du style
         - pas besoin de prendre latent_y_dim pour connaître la dimension de l'objectif si l'architecture
         est adaptée en fonction de la dimension de départ init_dim (voir juste en dessous)
        '''
        super().__init__()
        self.z_dim = z_dim # dimension du vecteur bruit 

        # dimensions des espaces X et Y (on suppose dim(X) < dim(Y))
        # G opère donc une déconvolution pour passer de X à Y
        self.x_dim = latent_x_dim # dimension du vecteur condition (espace BDs X)

        self.init_dim  = init_dim # dimension (H ou W) de l'image 2D à l'entrée du CNN
        # idéalement dim(Y) = dim_init * 2**(n_upsample) pour qu'on puisse facilement arriver à la bonne dimension (ici n_upsample = 2) 
        self.num_ch = x_ch # nombre de canaux arbitraire (dépend de l'espace latent qu'on choisit, peut être des autres conditions aussi)
        
        # Première couche linéaire
        self.l1 = nn.Sequential( 
            nn.Linear(self.z_dim + self.x_dim, self.num_ch * self.init_dim * self.init_dim) 
        )

        # Réseau convolutionnel --> déconvolution d'une petite dimension vers une grande dimension
        self.model = nn.Sequential(
            nn.BatchNorm2d(self.num_ch),
            nn.Upsample(scale_factor=2), # (init_dim)x(init_dim) --> (2*init_dim)x(2*init_dim)
            nn.Conv2d(self.num_ch, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2), # (2*init_dim)x(2*init_dim) --> (4*init_dim)x(4*init_dim)
            nn.Conv2d(256, 1, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )

    def forward(self, x, z):
        # Concaténation sur la dimension 1
        cat_xz = torch.cat([x, z], dim=1) 
        output1 = self.l1(cat_xz) 
        
        output2 = output1.view(output1.shape[0], self.num_ch, self.init_dim, self.init_dim) 

        output = self.model(output2) 
        
        return output

In [19]:
z_dim = 100 # arbitraire
x_dim = 100 # dimension d'un vecteur
x_ch = 32
y_dim = 64 # arbitraire, tenseur 2D 64x64
init_dim = 64 // (2**2)
G = Generator_G(z_dim, x_dim, x_ch, init_dim).to(device=device)

### Générateur $F$ : $Y$(audios) $\longrightarrow$ $X$(BDs)

Contrairement à $G$, qui développait les données, $F$ a pour but de compresser l'espace $Y$ vers $X$. On veut que les deux générateurs forment un équilibre lors de l'apprentissage. On a également les même dimensions que dans $G$, mais inversées. Il es dont logique que l'architecture de $F$ soit symétrique à celle de $G$.

In [20]:
class Generator_F(nn.Module):
    def __init__(self, z_dim, latent_x_dim, x_ch, y_ch, init_dim):
        '''
        Inputs :
         - latent_x_dim : dimension du vecteur de sortie (espace X)
         - latent_y_dim : nombre de canaux de l'espace latent d'entrée Y
         - z_dim        : dimension du bruit gaussien ajouté
         - init_dim     : dimension spatiale de référence
        '''
        super().__init__()
        self.x_dim = latent_x_dim
        self.y_ch = y_ch
        self.z_dim = z_dim     
        self.init_dim = init_dim
        self.internal_ch = x_ch

        # Le réseau prend maintenant en entrée canaux de Y + canaux de Z
        input_channels = self.y_ch + self.z_dim

        self.model = nn.Sequential(
            
            nn.Conv2d(input_channels, 256, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            
            # equivalent à un Downsample
            nn.AvgPool2d(kernel_size=2, stride=2), 

            nn.Conv2d(256, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.AvgPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, self.internal_ch, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(self.internal_ch),
            nn.LeakyReLU(0.2, inplace=True),
        )

        self.flat_dim = self.internal_ch * self.init_dim * self.init_dim
        
        self.final_head = nn.Sequential(
            nn.Linear(self.flat_dim, self.x_dim)
        )

    def forward(self, input_y, z):

        # comme on ajoute z à une matrice 2D, il faut l'étendre (comme la condition X dans DY)
        batch_size, _, h, w = input_y.size()
        z_expanded = z.view(batch_size, -1, 1, 1).expand(batch_size, -1, h, w)
        combined_input = torch.cat([input_y, z_expanded], dim=1)
        
        features = self.model(combined_input)
        features_flat = features.view(features.shape[0], -1)
        output_x = self.final_head(features_flat)
        
        return output_x

In [21]:
z_dim = 100 # arbitraire
x_dim = 100 # dimension d'un vecteur
x_ch = 32
y_ch = 64 # arbitraire, tenseur 2D 64x64
init_dim = 64 // (2**2)
F = Generator_F(z_dim, x_dim, x_ch, x_ch, init_dim).to(device=device)

### Discriminateur $D_Y$

$D_Y$ vérifie que $G(x) = \hat{y} \approx y$. 

**Au final les discriminateurs ne prennent pas de conditions. $\\$ EXPLICATION :  la loss des discriminateurs est calculée d'abord sur des prédictions à partir d'entrées totalement vraies, puis sur des prédictions à partir d'entrées totalement fausses (ou inversement) le conditionnement fonctionne pour la prédiction à partir de données fausses, puisque l'on veut que  les données générées depuis l'autre espace soit notées en fonction de l'autre espace. Cependant, pour les prédictions à partir de données réelles, comme les données ne sont pas pairées, la conditions ne veut rien dire pour le discriminateur. Il ne veut donc même pas reconnaître ça comme une donnée réelle lors du calcul de la loss (à clarifier).**

**Du coup, *Comment assurer que le modèle découvre une cohérence entre X et Y ?* On s'appuie pour cela sur la Cycle-loss, qui permet de retrouver les données d'un espace en appliquant les deux générations successives. Cette Loss oblige le modèle à lier les échantillons $x$ et $y$ qui lui permettront de reproduire le plus fidèlement les espaces à partir des générations (à clarifier)**. 

Le discriminateur va donc prendre en entrée des données 2D, on peut donc utiliser une structure patchGAN $N\times N$, qui permettra de réduire la quantité de calculs de la discrimination, et d'avoir un espace latent $Y$ plus complexe.

Un discriminateur de type PatchGAN ne sort pas un scalaire de prédiction mais une grille de prédiction pour chaque patche de données. La couche linéaire à la fin du discriminateur conditionnel vu juste avant doit donc nécessairement disparaître car on veut une sortie 2D pour le discriminateur.

In [22]:
class Discriminator_DY(nn.Module):
    def __init__(self, y_ch, ndf=64):
        """
        y_ch : Canaux de l'image Y
        ndf  : Nombre de filtres de base
        """
        super().__init__()

        self.model = nn.Sequential(
            # Layer 1 : Y -> Features
            nn.Conv2d(y_ch, ndf, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf, ndf * 2, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 2, ndf * 4, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            # Layer 4 : Sortie PatchGAN (Grille 2D)
            nn.Conv2d(ndf * 4, 1, kernel_size=4, stride=1, padding=1)
        )

    def forward(self, y_input):
        # Plus besoin de x_condition
        return self.model(y_input)

In [23]:
x_dim = 100 # dimension d'un vecteur
num_ch_y = 3 # arbitraire
DY = Discriminator_DY(num_ch_y).to(device=device)

### Discriminateur $D_X$

Ce discriminateur agit sur l'espace $X$. Les données sont donc 1D, et plus simples que pour $D_Y$, on n'a pas besoin d'utiliser un discriminateur de type PatchGAN. On utilise un discriminateur conditionnée classique.

In [ ]:
class Discriminator_DX(nn.Module):
    def __init__(self, x_ch, latent_x_dim, ndf=64):
        """
        x_ch         : Nombre de canaux de l'espace latent X (input)
        latent_x_dim : Dimension spatiale (H ou W) de l'entrée X
                       (Utilisé pour calculer la taille du vecteur aplati)
        ndf          : Nombre de filtres de base
        """
        super().__init__()

        self.x_ch = x_ch 
        self.stride_size = 2 

        self.model = nn.Sequential(
            nn.Conv2d(self.x_ch, 64, kernel_size=4, stride=2, padding=1), 
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), 
            nn.BatchNorm2d(128), 
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(128, 512, kernel_size=3, stride=2, padding=1), 
            nn.BatchNorm2d(512), 
            nn.LeakyReLU(0.2),
        )

        # Calcul de la dimension spatiale finale avant aplatissement
        # On divise par 8 car il y a 3 couches avec stride 2 (2^3 = 8)
        self.final_spatial_dim = int(latent_x_dim / 8)
        
        # Dimension du vecteur aplati
        self.flat_dim = 512 * self.final_spatial_dim * self.final_spatial_dim

        self.classifier = nn.Sequential(
            # Plus de cond_dim ici, seulement les features extraites
            nn.Linear(self.flat_dim, 1),
            #nn.Sigmoid() # pas de Sigmoid pour un WGAN
        )

    def forward(self, x):

        features = self.model(x)
        features_flat = features.view(features.size(0), -1) 
        output = self.classifier(features_flat)
        
        return output

In [25]:
x_dim = 100 # dimension d'un vecteur
y_dim = 28 # arbitraire, 28x28
num_ch_x = 3 # arbitraire
DX = Discriminator_DX(num_ch_x, x_dim).to(device=device)

### Entraînement du modèle

Pour entraîner nos deux générateurset nos discriminateurs, il faut successivement générerles données, et les prédictions et calculer les loss de chaque sous-structure à chaque itération.

In [26]:
# Datasets générés à partir de l'espace latent audio

batch_size = 32
train_loader_X = ???
train_loader_Y = ???
iter_Y = iter(train_loader_Y) # si train_loader_Y est plus petit, on itère quand on arrive à la fin

SyntaxError: invalid syntax (1530591102.py, line 4)

In [29]:
# Paramètres de l'entraînement

lr = 0.0001
num_epochs = 50
lambda_cycle = 0.2 # poids de la cycle_loss
lambda_gp = 10 # poids de la gradient penalty
n_critic = 5 # ration entraînement discriminateurs/générateurs
b1 = 0.0  # paramètres ADAM recommandés par le papier
b2 = 0.9

# Losses
# les losses des générateurs et discriminateurs pour un WGAN font simplement intervenir des torch.mean
loss_func_cycle = nn.BCELoss() 

optimizer_G = torch.optim.Adam(G.parameters(), lr=lr, betas=(b1,b2))
optimizer_F = torch.optim.Adam(F.parameters(), lr=lr, betas=(b1,b2))
optimizer_DY = torch.optim.Adam(DY.parameters(), lr=lr, betas=(b1,b2))
optimizer_DX = torch.optim.Adam(DX.parameters(), lr=lr, betas=(b1,b2))


In [ ]:
for epoch in range(num_epochs):
    for i, real_x in enumerate(train_loader_X):
        
        try:
            real_y = next(iter_Y)
        except StopIteration:
            # Si on a fini le dataset Y, on le recharge
            iter_Y = iter(train_loader_Y)
            real_y = next(iter_Y)

        real_x = real_x.to(device)
        real_y = real_y.to(device)
        
        # on s'assure qu'ils aient les bonnes dimensions
        real_x = ...
        real_y = ...

        with torch.no_grad(): # on ne veut que les gradients des générateurs soient calculés pour l'instant
            z_x = torch.randn(batch_size, 100).to(device)
            fake_x = F(real_y, z_y)
            z_y = torch.randn(batch_size, 100).to(device)
            fake_y = G(real_x, z_x)

        ### ENTRAINEMENT DISCRIMINATEURS
        optimizer_DY.zero_grad()
        optimizer_DX.zero_grad()
        
        # Discriminateur DY
        score_real_Y = DY(real_y) # les discriminateurs sortent des scores, pas des prédictions
        score_fake_Y = DY(fake_y.detach()) # detach() --> on ne veut pas que le gradient remonte au générateur
        # Loss WGAN : E[D(fake)] - E[D(real)] + lambda * GP
        loss_DY = torch.mean(score_fake_Y) - torch.mean(score_real_Y) + lambda_gp * compute_gradient_penalty(DY, real_y,fake_y.detach())
        loss_DY.backward()
        optimizer_DY.step()

        # Discriminateur DX
        score_real_X = DX(real_x) # les discriminateurs sortent des scores, pas des prédictions
        score_fake_X = DY(fake_x.detach()) # detach() --> on ne veut pas que le gradient remonte au générateur
        # Loss WGAN : E[D(fake)] - E[D(real)] + lambda * GP
        loss_DX = torch.mean(score_fake_X) - torch.mean(score_real_X) + lambda_gp * compute_gradient_penalty(DX, real_x,fake_x.detach())
        loss_DX.backward()
        optimizer_DX.step()

        ### ENTRAINEMENT GENERATEURS
        if i % n_critic:
            optimizer_G.zero_grad()
            optimizer_F.zero_grad()

            z_x = torch.randn(batch_size, 100).to(device)
            fake_x = F(real_y, z_y)
            z_y = torch.randn(batch_size, 100).to(device)
            fake_y = G(real_x, z_x)

            # Calcul des GAN-Loss --> tromper le discriminateur
            # Le générateur veut maximiser D(fake), donc minimiser -D(fake)
            predict_fake_Y = DY(fake_y)
            Loss_GAN_G = -torch.mean(predict_fake_Y)
            predict_fake_X = DX(fake_x)
            Loss_GAN_F = -torch.mean(predict_fake_X)

            # Calcul de Cycle-Loss --> pour revenir à l'espace de départ
            reconstructed_x = F(fake_y, torch.randn_like(z_y))
            Loss_cycle_X = loss_func_cycle(reconstructed_x, real_x)
            reconstructed_y = G(fake_x, torch.randn_like(z_x))
            Loss_cycle_Y = loss_func_cycle(reconstructed_y, real_y)

            # Loss totale
            Loss_Generator = Loss_GAN_G + Loss_GAN_F + lambda_cycle*(Loss_cycle_X + Loss_cycle_Y)
            Loss_Generator.backward()
            optimizer_G.step()
            optimizer_F.step()

        # Show loss
        if n == batch_size - 2:
            print(f"Epoch: {epoch} Loss G : {Loss_GAN_G}")
            print(f"Epoch: {epoch} Loss F : {Loss_GAN_F}")
            print(f"Epoch: {epoch} Loss DY : {loss_DY}")
            print(f"Epoch: {epoch} Loss DX : {loss_DX}")


NameError: name 'train_loader_X' is not defined